# Information Retrieval project
**Authors:** Arduini L., Menchini L., Namaki Ghaneh D., Petruzzella C.

**Dataset:** The chosen dataset is MSMARCO Passage dataset ()

**Evaluation:** For evaluation the trec-2020-dl dataset has been used 

# 0. Setup environment and dependencies
This section ensures that all necessary packages are installed and loaded.

**Note:** The project uses `ir_datasets`, `nltk`, and `ir_measures`, along with several utilities for processing.

In [1]:
!pip install ir_datasets
!pip install nltk
!pip install ir_measures
!pip install snowballstemmer


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import ir_datasets
import ir_measures
from ir_measures import *
import random
import re
import string
import nltk
import time
from collections import Counter, defaultdict
from tqdm.auto import tqdm
import gzip
import pickle
import os
import heapq
import math

/Users/krys/.pyenv/versions/3.12.0/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Loading the dataset

This notebook will load the MS MARCO Passage dataset, a standard dataset for Information Retrieval tasks.
It contains passages from various sources and is used to train and evaluate retrieval models.

For testing purposes, also the Vaswani dataset will be used in a development environment.

In [ ]:
# ------- Production Environment -------
dataset = ir_datasets.load("msmarco-passage")
# ---------------------------------------

# ------- Development Environment -------
# dataset = ir_datasets.load("vaswani")
# ---------------------------------------

# 2. Preprocessing text data
This section defines functions for text preprocessing. Preprocessing steps include:
- Lowercasing
- Replacing symbols and punctuations
- Removing stopwords
- Stemming tokens

The goal is to normalize text data for effective retrieval

In [4]:
from functools import lru_cache
import snowballstemmer
nltk.download("stopwords", quiet=True)

# ------- Pre Initialization -------
# 1. Compile regex patterns once globally
# 2. Preload stopwords set
# 3. Initialize stemmer

ACRONYM_REGEX = re.compile(r"(?<!\w)\.(?!\d)")
PUNCTUATION_TRANS = str.maketrans("", "", string.punctuation)
STOPWORDS = set(nltk.corpus.stopwords.words('english'))
STEMMER = snowballstemmer.stemmer('english')

# Define a cached function to stem individual words
@lru_cache(maxsize=1000)
def stem(word):
    return STEMMER.stemWord(word)

# ----------------------------------

def preprocess(s):
    """
    Preprocess a string for indexing or querying.

    Args:
        s: The input string.

    Returns:
        A list of preprocessed tokens.    
    """

    s = s.lower()
    s = s.replace("&", " and ")
    # normalize quotes and dashes
    s = s.translate(str.maketrans("‘’´“”–-", "'''\"\"--"))
    # remove unnecessary dots in acronyms (but not decimals)
    s = ACRONYM_REGEX.sub("", s)
    # remove punctuation
    s = s.translate(PUNCTUATION_TRANS)
    # strip and remove extra spaces
    s = " ".join(s.split())

    tokens = s.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    # Apply cached stemming function
    tokens = [stem(t) for t in tokens]

    return tokens

In [5]:
def profile(f):
    """
    A decorator that prints the runtime of the decorated function.

    Args:
        f: The function to profile.

    Returns:
        The profiled function.
    """
    def f_timer(*args, **kwargs):
        """
        The profiled function.
        
        Args:
            *args: The arguments to the function.
            **kwargs: The keyword arguments to the function.
            
        Returns:
            The result of the function.
        """
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

# 3. Building the inverted index
We create an inverted index to store terms with their respective document IDs and term frequencies.
The `build_index` function processes the dataset and constructs a structure that enables efficient term-based searching across documents

In [6]:
@profile
def build_index(dataset):
    """
    Build an inverted index from a dataset.

    Args:
        dataset: The dataset to index.

    Returns:
        A tuple of:
        - The lexicon, a dictionary mapping terms to term IDs and document frequencies.
        - The inverted index, a dictionary mapping term IDs to lists of document IDs and frequencies.
        - The document index, a list of document IDs and document lengths.
        - The index statistics, a dictionary of statistics.
    """
    lexicon = {}
    doc_index = []
    inv_d, inv_f = {}, {}
    termid = 0

    num_docs = 0
    total_dl = 0
    total_toks = 0
    for docid, doc in tqdm(enumerate(dataset.docs_iter()), desc='Indexing', total=dataset.docs_count()):
        tokens = preprocess(doc.text)
        token_tf = Counter(tokens)
        for token, tf in token_tf.items():
            if token not in lexicon:
                lexicon[token] = [termid, 0, 0]
                inv_d[termid], inv_f[termid] =  [], []
                termid += 1
            token_id = lexicon[token][0]
            inv_d[token_id].append(docid)
            inv_f[token_id].append(tf)
            lexicon[token][1] += 1
            lexicon[token][2] += tf
        doclen = len(tokens)
        doc_index.append((str(doc.doc_id), doclen))
        total_dl += doclen
        num_docs += 1


    stats = {
        'num_docs': 1 + docid,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [7]:
lex, inv, doc, stats = None, None, None, None

files = ['lexicon.pickle.gz', 'inverted_file.pickle.gz', 'document_index.pickle.gz', 'stats.pickle.gz']
if all(os.path.exists(file) for file in files):
    print("All files already exist.")
    
    for file, var_name in zip(files, ['lex', 'inv', 'doc', 'stats']):
        try:
            if os.path.getsize(file) > 0:  # Verifica se il file non è vuoto
                with gzip.open(file, 'rb') as f:
                    globals()[var_name] = pickle.load(f)
            else:
                print(f"Warning: {file} is empty.")
        except EOFError:
            print(f"Error: {file} is corrupted or incomplete. Rebuilding the index.")
            lex, inv, doc, stats = build_index(dataset)
            break
else:
    # Se i file non esistono o sono corrotti, ricostruisci l'indice
    lex, inv, doc, stats = build_index(dataset)

    # Salva nuovamente i dati nei file compressi solo se necessario
    for data, file in zip([lex, inv, doc, stats], files):
      with gzip.open(file, 'wb') as f:
        print(f"Saving {file}...")
        pickle.dump(data, f)


Indexing: 100%|██████████| 11429/11429 [00:01<00:00, 6530.89it/s]


build_index (1844.108 ms)
Saving lexicon.pickle.gz...
Saving inverted_file.pickle.gz...
Saving document_index.pickle.gz...
Saving stats.pickle.gz...


In [8]:
class InvertedIndex:
    """
    A simple inverted index class.
    
    Attributes:
        lexicon: The lexicon.
        inv: The inverted index.
        doc: The document index.
        stats: The index statistics.
        
    Methods:
        num_docs: Get the number of documents in the index.
        get_posting: Get a posting list iterator for a term.
        get_termids: Get the term IDs for a list of tokens.
        get_postings: Get the posting list iterators for a list of term IDs.
        
    Inner class:
        PostingListIterator: An iterator over a posting list.
    """

    class PostingListIterator:
        """
        An iterator over a posting list.

        Attributes:
            docids: The list of document IDs.
            freqs: The list of term frequencies.
            pos: The current position in the posting list.
            doc: The document index.

        Methods:
            docid: Get the current document ID.
            score: Get the current document score.
            next: Move to the next document.
            is_end_list: Check if the iterator is at the end of the list.
            len: Get the length of the posting list.
        """
        def __init__(self, docids, freqs, doc):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc

        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]

        def score(self):
            if self.is_end_list():
                return math.inf
            return self.freqs[self.pos]/self.doc[self.docid()][1]

        def next(self, target = None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    try:
                        self.pos = self.docids.index(target, self.pos)
                    except ValueError:
                        self.pos = len(self.docids)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)


    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stat = stats

    def num_docs(self):
        return self.stats['num_docs']

    def get_posting(self, termid):
        return InvertedIndex.PostingListIterator(self.inv['docids'][termid], self.inv['freqs'][termid], self.doc)

    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]
    
inv_index = InvertedIndex(lex, inv, doc, stats)

# 4. Query processing
This section implements the Query Processing task, aiming to rank documents by relevance to a given query using the BM25 and TF-IDF scoring algorithm with Document-at-a-Time (DAAT) and Term-at-a-Time (TAAT) approaches.

In [ ]:
# ------- Production Environment --------
trec_dl_2020 = ir_datasets.load("msmarco-passage/trec-dl-2020")
# ---------------------------------------

# ------- Development Environment -------
# trec_dl_2020 = ir_datasets.load("vaswani")
# ---------------------------------------

In [10]:
class TopQueue:
    """
    A simple top-k queue class.
    
    Attributes:
        queue: The priority queue.
        k: The maximum number of items in the queue.
        threshold: The minimum score threshold.
        
    Methods:
        size: Get the number of items in the queue.
        would_enter: Check if a score would enter the queue.
        clear: Clear the queue.
        insert: Insert a document into the queue.
    """
    def __init__(self, k=10, threshold=0.0):
        self.queue = []
        self.k = k
        self.threshold = threshold

    def size(self):
        return len(self.queue)

    def would_enter(self, score):
        return score > self.threshold

    def clear(self, new_threshold=None):
        self.queue = []
        if new_threshold:
            self.threshold = new_threshold

    def __repr__(self):
        return f'<{self.size()} items, th={self.threshold} {self.queue}'

    def insert(self, docid, score):
        if score > self.threshold:
            if self.size() >= self.k:
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True
        return False

## 4.1. BM25

In [11]:
# Average document length
avg_dl = inv_index.stat['num_tokens'] / inv_index.stat['num_docs']
# Number of documents
N = inv_index.stat['num_docs']

def bm25(tf, df, dl, k1=1.5, b=0.75):
    """
    Compute the BM25 score.

    Args:
        tf: The term frequency.
        df: The document frequency.
        dl: The document length.
        k1: The k1 parameter.
        b: The b parameter.

    Returns:
        The BM25 score.
    """
    idf = math.log(1 + (N - df + 0.5) / (df + 0.5))
    term_frequency_component = (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (dl / avg_dl)))
    return idf * term_frequency_component

### 4.1.1 DAAT with BM25

In [12]:
# Calculate document lengths
doc_lengths = defaultdict(int)
for docid, doc_len in inv_index.doc:
    doc_lengths[docid] = doc_len

def min_docid(postings):
    """
    Get the minimum document ID from a list of posting list iterators.
    
    Args:
        postings: The list of posting list iterators.
        
    Returns:
        The minimum document ID.
    """
    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat_bm25(postings, k=10):
    """
    Perform a document-at-a-time (DAAT) scoring using BM25.

    Args:
        postings: The list of posting list iterators.
        k: The maximum number of documents to retrieve.
    
    Returns:
        A list of (docid, score) tuples.
    """
    top = TopQueue(k)
    current_docid = min_docid(postings)

    while current_docid != math.inf:
        score = 0
        next_docid = math.inf

        for posting in postings:
            if posting.docid() == current_docid:
                tf = posting.freqs[posting.pos]
                df = posting.len()
                dl = doc_lengths[current_docid]

                score += bm25(tf, df, dl)

                posting.next()
            if not posting.is_end_list():
                next_docid = min(next_docid, posting.docid())

        top.insert(current_docid, score)
        current_docid = next_docid

    return sorted(top.queue, reverse=True)

### 4.1.2 TAAT with BM25

In [13]:
def taat_bm25(postings, k=10):
    """
    Perform a term-at-a-time (TAAT) scoring using BM25.

    Args:
        postings: The list of posting list iterators.
        k: The maximum number of documents to retrieve.
    
    Returns:
        A list of (docid, score) tuples.
    """
    A = defaultdict(float)

    for posting in postings:
        current_docid = posting.docid()

        df = posting.len()

        while current_docid != math.inf:
            tf = posting.freqs[posting.pos]
            dl = doc_lengths[current_docid]

            score = bm25(tf, df, dl)
            A[current_docid] += score

            posting.next()
            current_docid = posting.docid()

    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)

    return sorted(top.queue, reverse=True)

## 4.2 TF-IDF

In [14]:
def tfidf_score(tf, df, N):
    """
    Compute the TF-IDF score.
    
    Args:
        tf: The term frequency.
        df: The document frequency.
        N: The number of documents.
        
    Returns:
        The TF-IDF score.
    """
    idf = math.log(N / df)
    return tf * idf

### 4.2.1 DAAT with TF-IDF

In [15]:
def daat_tfidf(postings, k=10):
    """
    Perform a document-at-a-time (DAAT) scoring using TF-IDF.

    Args:
        postings: The list of posting list iterators.
        k: The maximum number of documents to retrieve.
    
    Returns:
        A list of (docid, score) tuples.
    """
    top = TopQueue(k)
    current_docid = min_docid(postings)

    while current_docid != math.inf:
        score = 0
        next_docid = math.inf

        for posting in postings:
            if posting.docid() == current_docid:
                tf = posting.freqs[posting.pos]
                df = posting.len() 
                
                score += tfidf_score(tf, df, N)
                
                posting.next()
            if not posting.is_end_list():
                next_docid = min(next_docid, posting.docid())

        top.insert(current_docid, score)
        current_docid = next_docid

    return sorted(top.queue, reverse=True)


### 4.2.2 TAAT with TF-IDF

In [16]:
def taat_tfidf(postings, k=10):
    """
    Perform a term-at-a-time (TAAT) scoring using TF-IDF.

    Args:
        postings: The list of posting list iterators.
        k: The maximum number of documents to retrieve.
    
    Returns:
        A list of (docid, score) tuples.
    """
    A = defaultdict(float)
    
    for posting in postings:
        current_docid = posting.docid()
        
        df = posting.len()
        
        while current_docid != math.inf:
            tf = posting.freqs[posting.pos]
            
            score = tfidf_score(tf, df, N)
            A[current_docid] += score
            
            posting.next()
            current_docid = posting.docid()
    
    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)

    return sorted(top.queue, reverse=True)


## 4.3 Results

In [17]:
@profile
def query_processing(queries_iter, fn):
    """
    Process a list of queries using a scoring function.

    Args:
        queries_iter: The list of queries.
        fn: The scoring function.
    
    Returns:
        A list of query results.
    """
    
    res = []
    for q in queries_iter:
        query = preprocess(q.text)
        termids = inv_index.get_termids(query)
        postings = inv_index.get_postings(termids)
        res.append({'query_id': q.query_id, 'scores': fn(postings)})
    return res

In [18]:
print(query_processing(trec_dl_2020.queries_iter(), daat_bm25))

query_processing (3797.972 ms)
[{'query_id': '1', 'scores': [(28.57672857738475, 5501), (28.366170176908092, 7233), (27.18674124075531, 8171), (25.547677818495405, 6823), (24.759597953363038, 9858), (24.426051614774867, 4568), (22.938509185266778, 719), (22.856011517307113, 7922), (22.350366336092655, 2235), (21.864580447315486, 7733)]}, {'query_id': '2', 'scores': [(26.882886822105817, 8252), (25.190452775793243, 4340), (24.64283112962301, 8240), (24.22332191639271, 5638), (22.201849094099728, 5123), (21.497673270474923, 2283), (20.513925176734382, 7112), (20.513925176734382, 1373), (20.18305091333257, 2728), (20.18305091333257, 2217)]}, {'query_id': '3', 'scores': [(47.40019567209051, 6347), (47.07774749796472, 7085), (42.83499514919195, 6535), (41.450461861928815, 5044), (40.11492511651749, 6973), (39.81561685529442, 5975), (39.39303404401454, 9288), (38.06872302348601, 8135), (37.35269069056093, 4724), (37.27322919502089, 8023)]}, {'query_id': '4', 'scores': [(27.337301259114952, 4

In [19]:
bm25_results = query_processing(trec_dl_2020.queries_iter(), taat_bm25)
print(bm25_results)

query_processing (909.090 ms)
[{'query_id': '1', 'scores': [(28.57672857738475, 5501), (28.366170176908092, 7233), (27.18674124075531, 8171), (25.547677818495405, 6823), (24.759597953363038, 9858), (24.426051614774867, 4568), (22.938509185266778, 719), (22.856011517307113, 7922), (22.350366336092655, 2235), (21.864580447315486, 7733)]}, {'query_id': '2', 'scores': [(26.882886822105817, 8252), (25.190452775793243, 4340), (24.64283112962301, 8240), (24.22332191639271, 5638), (22.201849094099728, 5123), (21.497673270474923, 2283), (20.513925176734382, 7112), (20.513925176734382, 1373), (20.18305091333257, 2728), (20.18305091333257, 2217)]}, {'query_id': '3', 'scores': [(47.40019567209051, 6347), (47.07774749796472, 7085), (42.83499514919195, 6535), (41.450461861928815, 5044), (40.11492511651749, 6973), (39.81561685529442, 5975), (39.39303404401454, 9288), (38.06872302348601, 8135), (37.35269069056093, 4724), (37.27322919502089, 8023)]}, {'query_id': '4', 'scores': [(27.337301259114952, 45

In [20]:
print(query_processing(trec_dl_2020.queries_iter(), daat_tfidf))

query_processing (2586.144 ms)
[{'query_id': '1', 'scores': [(30.223860516119117, 3692), (29.691583309945827, 7233), (25.41315778137699, 5501), (24.32090058992211, 9858), (23.273173682113566, 7922), (21.529062765292075, 4570), (20.233390332013307, 7733), (19.616342494992008, 2235), (19.225903771406433, 6823), (19.07435886632644, 4571)]}, {'query_id': '2', 'scores': [(27.31887911071231, 3333), (22.43881142618663, 4570), (20.862578575260166, 8252), (20.723962639433328, 8996), (20.490238869696903, 7777), (18.537348166556495, 10234), (17.67696990561112, 4340), (17.56306188831163, 8380), (17.56306188831163, 4135), (17.098382867181424, 8382)]}, {'query_id': '3', 'scores': [(57.71381165880253, 7085), (55.462490065018585, 6347), (54.61231741908634, 5649), (51.397655071546964, 6973), (48.01943578981415, 6535), (46.66208107908988, 5978), (44.449523170986595, 5249), (40.374052284394494, 8345), (40.10101534683967, 9179), (36.008923306134974, 4708)]}, {'query_id': '4', 'scores': [(62.94518851425234

In [21]:
tfidf_results = query_processing(trec_dl_2020.queries_iter(), taat_tfidf)
print(tfidf_results)

query_processing (674.054 ms)
[{'query_id': '1', 'scores': [(30.223860516119117, 3692), (29.691583309945827, 7233), (25.41315778137699, 5501), (24.32090058992211, 9858), (23.273173682113566, 7922), (21.529062765292075, 4570), (20.233390332013307, 7733), (19.616342494992008, 2235), (19.225903771406433, 6823), (19.07435886632644, 4571)]}, {'query_id': '2', 'scores': [(27.31887911071231, 3333), (22.43881142618663, 4570), (20.862578575260166, 8252), (20.723962639433328, 8996), (20.490238869696903, 7777), (18.537348166556495, 10234), (17.67696990561112, 4340), (17.56306188831163, 8380), (17.56306188831163, 4135), (17.098382867181424, 8382)]}, {'query_id': '3', 'scores': [(57.71381165880253, 7085), (55.462490065018585, 6347), (54.61231741908634, 5649), (51.397655071546964, 6973), (48.01943578981415, 6535), (46.66208107908988, 5978), (44.449523170986595, 5249), (40.374052284394494, 8345), (40.10101534683967, 9179), (36.008923306134974, 4708)]}, {'query_id': '4', 'scores': [(62.94518851425234,

# 5. Evaluation with TREC-style measures
To evaluate retrieval performance, we use the TREC evaluation method with `ir_measures`.

This section generates a run file and QRELs for the TREC evaluation tool.

In [23]:
for query in list(trec_dl_2020.queries_iter())[:3]:
    print(query)

GenericQuery(query_id='1', text='MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES\n')
GenericQuery(query_id='2', text='MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS\n')
GenericQuery(query_id='3', text='USE OF DIGITAL COMPUTERS IN THE DESIGN OF BAND PASS FILTERS HAVING GIVEN PHASE AND ATTENUATION CHARACTERISTICS\n')


In [24]:
for ass in list(trec_dl_2020.qrels_iter())[:3]:
  print(ass)

TrecQrel(query_id='1', doc_id='1239', relevance=1, iteration='0')
TrecQrel(query_id='1', doc_id='1502', relevance=1, iteration='0')
TrecQrel(query_id='1', doc_id='4462', relevance=1, iteration='0')


## 5.1 Run File generation

In [25]:
def generate_run(results):
    trec_run_list = []
    for doc_scores in results:
        rank = 1
        query_id = doc_scores['query_id']
        scores = doc_scores['scores']

        for score, doc_id in scores:
            line = f"{query_id} Q0 {doc_id} {rank} {score} GOODFELLAS"
            trec_run_list.append(line)
            rank += 1
    
    return trec_run_list

trec_bm25_run_list = generate_run(bm25_results)
trec_tfidf_run_list = generate_run(tfidf_results)

with open("trec_eval_bm25_run_file.txt", "w") as f:
    for line in trec_bm25_run_list:
        f.write(line + "\n")

with open("trec_eval_tfidf_run_file.txt", "w") as f:
    for line in trec_tfidf_run_list:
        f.write(line + "\n")

## 5.2 Qrels File generation

In [26]:
# Create format for Trec_Eval
qrels_file = []
for qrel in trec_dl_2020.qrels_iter():
    line = f"{qrel.query_id} 0 {qrel.doc_id} {qrel.relevance}"
    qrels_file.append(line)

with open("trec_eval_qrels_file.txt", "w") as f:
    for line in qrels_file:
        f.write(line + "\n")

## 5.3 Results

In [27]:
measures = [P@5, P(rel=2)@5, nDCG@10, AP, AP(rel=2), Bpref, Bpref(rel=2), Judged@10]

qrels = ir_measures.read_trec_qrels('trec_eval_qrels_file.txt')
bm25_run = ir_measures.read_trec_run(('trec_eval_bm25_run_file.txt'))
bm25_results = ir_measures.calc_aggregate(measures, qrels, bm25_run)

qrels = ir_measures.read_trec_qrels('trec_eval_qrels_file.txt')
tfidf_run = ir_measures.read_trec_run(('trec_eval_tfidf_run_file.txt'))
tfidf_results = ir_measures.calc_aggregate(measures, qrels, tfidf_run)

In [28]:
import pandas as pd

# Create DataFrame for comparison
df = pd.DataFrame({
    "BM25": bm25_results,
    "TF-IDF": tfidf_results
})

print(df)

                  BM25    TF-IDF
Judged@10     0.033333  0.019355
P@5           0.027957  0.025806
AP(rel=2)     0.000000  0.000000
Bpref         0.012495  0.008701
AP            0.004410  0.002152
Bpref(rel=2)  0.000000  0.000000
P(rel=2)@5    0.000000  0.000000
nDCG@10       0.031989  0.021158
